<a href="https://colab.research.google.com/github/mansivai/ArrayString/blob/master/TEXT_CLASS_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
import re

Load the traing data set

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/ClauseClassification - ClauseClassification.csv' , names=["Traning set","class"], na_values="NaN")
data.head()

,Traning set,class
0,The Borrower and certain lenders and agents ar...,Other
1,This CREDIT AGREEMENT is entered into as of No...,Preamble
2,This agreement is dated 22 December 2014 and m...,Preamble
3,This Agreement and shall be governed by and in...,Governing Law
4,The choice of South African law as the governi...,Governing Law


In [4]:
uf = pd.read_csv('/content/drive/MyDrive/TextClassification_test - Sheet1.csv' , names=["Traning set"], na_values=" NaN")
uf.head()

,Traning set
0,"Amendment No. 1 dated as of February 4, 2016 (..."
1,"Subject to the Legal Reservations, (a) the cho..."
2,"This Agreement, and any non-contractual obliga..."
3,Each Lender agrees to indemnify the Administra...
4,Any Lender that (a) has failed to (i) fund all...


In [6]:
# training data
data=data.fillna('')
# test data
uf=uf.fillna('')


Remove stopwords

In [7]:
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize,word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
nltk.download('stopwords')
from nltk.corpus import stopwords

def remove_stopwords(x):
  x=word_tokenize(x)
  stopwords_all = stopwords.words('english')
  x = " ".join(i for i in x if not(i in stopwords_all))
  return x

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
data["Traning set"] = data["Traning set"].apply(remove_stopwords)
uf["Traning set"] = uf["Traning set"].apply(remove_stopwords)

In [11]:
def number_removal(row):
    data1 = row['Traning set']
    if type(data1) not in [int,float]:
        line = re.sub(r"[^A-Za-z\s]", " ", data1.strip())
        tokens = line.split()
    else:
        tokens=[]
    return ' '.join(tokens)
frequency_words_wo_stop = {}
def generate_word_frequency(row):
    data1 = row['Traning set']
    tokens = nltk.wordpunct_tokenize(data1)
    token_list = []
    for token in tokens:
        token_list.append(token.lower())
        if token.lower() in frequency_words_wo_stop:
            count = frequency_words_wo_stop[token.lower()]
            count = count + 1
            frequency_words_wo_stop[token.lower()] = count
        else:
            frequency_words_wo_stop[token.lower()] = 1
    return ','.join(token_list)


In [19]:
wordnet_lemmatizer = WordNetLemmatizer()

def receieve(query):
    data1=query
    data1['Traning set'] = data1.apply(number_removal,axis=1)
    data1['tokens'] = data1.apply(generate_word_frequency,axis=1)
    # print (data1.head())
    big=[]
    for i in data1['tokens']:
        st=''
        ls=[]
        for j in i.split(','):
            # print (wordnet_lemmatizer.lemmatize(j))
            ls.append(wordnet_lemmatizer.lemmatize(j))
        big.append(' '.join(ls))
    data1['Traning set_lem']=big
    return data1['Traning set_lem']
data['Traning set']=receieve(data[['Traning set']])
uf["Traning set"] = receieve(uf[["Traning set"]])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Model Training

In [20]:
class_names=['Preamble','Governing Law','Lender Defaulting','Indemnification' , 'Other']

In [21]:
data = data.sample(frac=1, random_state=15).reset_index(drop=True)

In [22]:
arr=data['Traning set'].tolist()
arr_lbl = data['class'].tolist()

In [23]:
train_arr = arr
test_arr = uf["Traning set"]
train_lbl  = arr_lbl

In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [25]:
vectorizer = CountVectorizer()
vectorizer.fit(train_arr)
train_mat = vectorizer.transform(train_arr)
print (train_mat.shape)
test_mat = vectorizer.transform(test_arr)
print (test_mat.shape)

(27, 269)
(10, 269)


In [26]:
tfidf = TfidfTransformer()
tfidf.fit(train_mat)
train_tfmat = tfidf.transform(train_mat)
print (train_tfmat.shape)
test_tfmat = tfidf.transform(test_mat)
print (test_tfmat.shape)

(27, 269)
(10, 269)


In [27]:
del train_arr

In [28]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from pprint import pprint
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix,accuracy_score

In [29]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix',cmap=plt.cm.Reds):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

#Naive Bayes 

Bernoulli Naive Bayes


In [46]:
bnb= BernoulliNB()
#fitting
bnb.fit(train_tfmat, train_lbl)
#ypred
ypredBnb = bnb.predict(test_tfmat)
print(ypredBnb)
uf["BernoulliNB"] = ypredBnb


['Preamble' 'Governing Law' 'Governing Law' 'Indemnification'
 'Lender Defaulting' 'Governing Law' 'Indemnification' 'Other' 'Other'
 'Lender Defaulting']


Multinomial Naive Bayes


In [49]:
mnb=MultinomialNB()
#fitting
mnb.fit(train_tfmat, train_lbl)
ypredMnb = mnb.predict(test_tfmat)
print(ypredMnb)
uf['MultinomialNB']=ypredMnb


['Preamble' 'Governing Law' 'Governing Law' 'Indemnification'
 'Lender Defaulting' 'Governing Law' 'Indemnification' 'Lender Defaulting'
 'Other' 'Lender Defaulting']


#SVM

In [35]:
from sklearn.svm import LinearSVC, NuSVC, SVC

Linear SVC

In [50]:
lsvm= LinearSVC()
#fitting
lsvm.fit(train_tfmat, train_lbl)
#ypred
ypredLsvm = lsvm.predict(test_tfmat)
print(ypredLsvm)
uf['LinearSVC']=ypredLsvm

['Preamble' 'Governing Law' 'Governing Law' 'Indemnification'
 'Lender Defaulting' 'Governing Law' 'Indemnification' 'Lender Defaulting'
 'Other' 'Lender Defaulting']


Non linear svm

In [51]:
nusvm=NuSVC()
#fitting
nusvm.fit(train_tfmat, train_lbl)
#ypred
ypredNusvm = nusvm.predict(test_tfmat)
print(ypredNusvm)
uf['NuSVC']=ypredNusvm

['Preamble' 'Governing Law' 'Governing Law' 'Indemnification'
 'Lender Defaulting' 'Governing Law' 'Indemnification' 'Lender Defaulting'
 'Other' 'Lender Defaulting']


#KNN

In [41]:
from sklearn.neighbors import KNeighborsClassifier

Uniform KNN

In [52]:
knn_uni = KNeighborsClassifier(n_neighbors=5, weights = 'uniform')
#fitting
knn_uni.fit(train_tfmat, train_lbl)
#ypred
ypred_knn_uni = knn_uni.predict(test_tfmat)
print(ypred_knn_uni)
uf['KNeighborsClassifier']=ypred_knn_uni

['Preamble' 'Governing Law' 'Governing Law' 'Indemnification'
 'Indemnification' 'Governing Law' 'Indemnification' 'Lender Defaulting'
 'Other' 'Lender Defaulting']


Distinct KNN

In [53]:
knn_dist = KNeighborsClassifier(n_neighbors=5, weights = 'distance')
#fitting
knn_dist.fit(train_tfmat, train_lbl)
#ypred
ypred_knn_dist = knn_dist.predict(test_tfmat)
print(ypred_knn_dist)

uf['KNeighborsClassifier_dist']=ypred_knn_dist

['Preamble' 'Governing Law' 'Governing Law' 'Indemnification'
 'Lender Defaulting' 'Governing Law' 'Lender Defaulting'
 'Lender Defaulting' 'Other' 'Lender Defaulting']


In [55]:
uf

,Traning set,BernoulliNB,MultinomialNB,LinearSVC,NuSVC,KNeighborsClassifier,KNeighborsClassifier_dist
0,amendment no dated february amendment no amend...,Preamble,Preamble,Preamble,Preamble,Preamble,Preamble
1,subject legal reservation choice law specified...,Governing Law,Governing Law,Governing Law,Governing Law,Governing Law,Governing Law
2,this agreement non contractual obligation aris...,Governing Law,Governing Law,Governing Law,Governing Law,Governing Law,Governing Law
3,each lender agrees indemnify administrative ag...,Indemnification,Indemnification,Indemnification,Indemnification,Indemnification,Indemnification
4,any lender failed fund portion loan within two...,Lender Defaulting,Lender Defaulting,Lender Defaulting,Lender Defaulting,Indemnification,Lender Defaulting
5,this assignment assumption shall governed cons...,Governing Law,Governing Law,Governing Law,Governing Law,Governing Law,Governing Law
6,a soon practicable payment indemnified tax oth...,Indemnification,Indemnification,Indemnification,Indemnification,Indemnification,Lender Defaulting
7,non defaulting lender mean time lender default...,Other,Lender Defaulting,Lender Defaulting,Lender Defaulting,Lender Defaulting,Lender Defaulting
8,fundco facility agreement mean facility agreem...,Other,Other,Other,Other,Other,Other
9,notwithstanding provision agreement contrary l...,Lender Defaulting,Lender Defaulting,Lender Defaulting,Lender Defaulting,Lender Defaulting,Lender Defaulting
